# Co-localization Analysis on Drosophila spreads

This notebook analyses co-localization on Drosophila salivary glands spreads

### Import Packages

In [8]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler_core.preferences as cpprefs
import cellprofiler.modules as cpm
import cellprofiler_core.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler_core.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
from omero.gateway import BlitzGateway

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

from getpass import getpass

### Set Cell Output Directory

In [9]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials and dataset to analyze

In [10]:
HOST = input("host: ") or 'omero.mri.cnrs.fr'
conn = BlitzGateway(input("OMERO Username: "),
                    getpass("OMERO Password: "),
                    group=input("OMERO Group: "),
                    host=HOST)
conn.connect()

dataset_id = input("Dataset id: ")
dataset = conn.getObject("Dataset", dataset_id)
print('Dataset Name: ',dataset.getName())

host: 
OMERO Username: mateos
OMERO Password: ········
OMERO Group: OMERO_training
Dataset id: 15657
Dataset Name:  Annotations and searching


### Load a cellprofiler pipeline attached to the dataset
Be careful, this script only takes the first pipeline that is attached to a dataset, so best keep only one pipeline in the dataset

In [13]:
anns = dataset.listAnnotations()
for ann in anns:
    if type(ann).__name__ == 'FileAnnotationWrapper' and ann.getFileName().endswith('.cppipe'):
        pipe_path = os.path.join(new_output_directory, ann.getFileName())
        with open(str(pipe_path), 'wb') as f:
            for chunk in ann.getFileInChunks():
                f.write(chunk)

pipeline = cpp.Pipeline()
pipeline.load(str(pipe_path))

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)
for i in range(4):
    print('Remove module: ', pipeline.modules()[9].module_name)
    pipeline.remove_module(10)

print('Pipeline modules:')
for module in pipeline.modules():
    print(module.module_num, module.module_name)

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Remove module:  OverlayOutlines
Remove module:  SaveImages
Remove module:  SaveImages
Remove module:  SaveImages
Pipeline modules:
1 RescaleIntensity
2 RescaleIntensity
3 CorrectIlluminationCalculate
4 CorrectIlluminationCalculate
5 CorrectIlluminationApply
6 IdentifyPrimaryObjects
7 MaskImage
8 IdentifyPrimaryObjects
9 IdentifyPrimaryObjects
10 MeasureColocalization
11 ExportToSpreadsheet


### Run Cell Profiler Pipeline on the OMERO Images

In [14]:
warnings.filterwarnings('ignore')

image_df = pandas.DataFrame()

for image in dataset.listChildren():

    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()

    channel_names = {'HA': 1,
                     'Pc': 2,
                     }

    for image_name, channel in channel_names.items():

        plane = pixels.getPlane(0, channel, 0)
        # image_name = image.getName()

        inject_image_module = InjectImage(image_name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    m = pipeline_copy.run()

    # Results obtained as CSV from Cell Profiler
    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv['Image Name'] = image.getName()
    # image_csv.ImageId = image.getId()
    image_df = image_df.append(image_csv)

## Results obtained as CSV from Cell Profiler

### Images (Summary)

In [15]:
image_df.head()

,Correlation_Costes_CorrHA_CorrPc,Correlation_Costes_CorrPc_CorrHA,Correlation_Manders_CorrHA_CorrPc,Correlation_Manders_CorrPc_CorrHA,Count_Mask_nonStrong_Pc,Count_PcBands_strong,Count_PcBands_weak,ExecutionTime_01InjectImage,ExecutionTime_02InjectImage,ExecutionTime_03RescaleIntensity,...,Threshold_OrigThreshold_Mask_nonStrong_Pc,Threshold_OrigThreshold_PcBands_strong,Threshold_OrigThreshold_PcBands_weak,Threshold_SumOfEntropies_Mask_nonStrong_Pc,Threshold_SumOfEntropies_PcBands_strong,Threshold_SumOfEntropies_PcBands_weak,Threshold_WeightedVariance_Mask_nonStrong_Pc,Threshold_WeightedVariance_PcBands_strong,Threshold_WeightedVariance_PcBands_weak,Image Name
0,0.348671,0.632989,0.024235,0.750707,1.0,27.0,246.0,0.00,0.0,0.01,...,0.0,0.4,0.1,0.0,-9.771072,-12.394296,1.734106,1.480816,1.539987,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...
0,0.074082,0.583960,0.020302,0.911975,2.0,32.0,211.0,0.00,0.0,0.01,...,0.0,0.4,0.1,0.0,-10.270415,-12.400549,1.665635,1.465389,1.478277,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...
0,1.000000,0.870056,0.023082,0.404146,1.0,28.0,224.0,0.01,0.0,0.01,...,0.0,0.4,0.1,0.0,-10.684191,-12.384959,1.794151,1.444759,1.633165,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...
0,1.000000,0.849891,0.025395,0.595204,1.0,32.0,367.0,0.01,0.0,0.01,...,0.0,0.4,0.1,0.0,-9.731390,-12.386205,1.918251,1.650731,1.696826,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...
0,1.000000,0.857828,0.030059,0.655601,1.0,48.0,301.0,0.00,0.0,0.01,...,0.0,0.4,0.1,0.0,-10.509497,-12.468774,1.842413,1.636007,1.580142,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...


### Push Results back to OMERO

In [16]:
file_to_upload = str(os.path.join(new_output_directory, 'Image.csv'))
image_df.to_csv(file_to_upload, index=False)

# create the original file and file annotation (uploads the file etc.)
namespace = "cellprofiler.analysis"
print("\nCreating an OriginalFile and FileAnnotation")
file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(),
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
dataset.linkAnnotation(file_ann)


Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 98408 , Image.csv Size: 16787


<FileAnnotationWrapper id=98408>

#### Close the connection to the OMERO server

In [17]:
conn.close()